# Exploring the CommaAI Device data

This requires python 3.11 or later because of the usage of the tools folder from Openpilot, which uses 3.11 features

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import pandas as pd
from pathlib import Path
# This is taken from the Openpilot project
from openpilot.tools.lib.logreader import LogReader, ReadMode
from dataclasses import dataclass
from datetime import datetime
from typing import List
import capnp
import json

In [2]:
# device_path = Path("/home/ulrikro/datasets/CommaAI/2024_01_14_TrondheimSnow/2024-01-14--13-01-26--10/")  # Old
# device_path = Path("/home/ulrikro/datasets/CommaAI/2024_02_28_Orkdal/2024-02-28--10-54-05--22/")  # New
device_path = Path("/home/ulrikro/datasets/CommaAI/2024_03_27_Are//2024-03-28--16-36-37--20/")  # New

# About the folder

Each folder, marked by their data and a number, contains 5 files: ecamera.hevc, fcamera.hevc, qcamera.ts, qlog, rlog. Each file contains 1 min video. 

* ecamera.hevc - road facing wide angle camera 1928*1208 at 20Hz
* fcamera.hevc - road facing narrow camera 1928*1208 at 20Hz
* qcamera.ts - driver facing wide angle camera 526*330 at 20Hz
* qlog - some
* rlog - some



In [3]:
ecamera_path = device_path / "ecamera.hevc"
fcamera_path = device_path / "fcamera.hevc"
qcamera_path = device_path / "qcamera.ts"
qlog_path = device_path / "qlog"
rlog_path = device_path / "rlog"


# Overview of the video files

In [4]:
def read_video_frames(file_path: str):
    frames = []
    video = cv2.VideoCapture(file_path)
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frames.append(frame)
    video.release()
    return frames

In [5]:
# ecamera_frames = read_video_frames(ecamera_path.as_posix())
# len(ecamera_frames)

In [6]:
# ecamera_frames[0].shape

In [7]:
# fcamera_frames = read_video_frames(fcamera_path.as_posix())
#len(fcamera_frames)

In [8]:
# fcamera_frames[0].shape

In [9]:
# qcamera_frames = read_video_frames(qcamera_path.as_posix())
# len(qcamera_frames)

In [10]:
# qcamera_frames[0].shape

# Overview of the log files

In [11]:


qlog_data = LogReader(qlog_path.as_posix(), default_mode=ReadMode.QLOG)

In [12]:
qlog_list = list(qlog_data)
len(qlog_list)

11367

In [13]:
type(qlog_list[0])

capnp.lib.capnp._DynamicStructReader

Each iteration throug the log data retruns a capnp.lib.capnp._DynamicStructReader object. 

capnp is a proto schema parsing library 

The proto schema seems to be defined in openpilot/common/params.cc
 


In [14]:
@dataclass
class CommandEntry:
    key: str
    value: str

@dataclass
class Commands:
    entries: List[CommandEntry]

@dataclass
class ParamEntry:
    key: str
    value: str

@dataclass
class Params:
    entries: List[ParamEntry]

@dataclass
class InitData:
    kernelArgs: List[str]
    dongleId: str
    deviceType: str
    version: str
    dirty: bool
    gitCommit: str
    gitBranch: str
    passive: bool
    gitRemote: str
    kernelVersion: str
    params: Params
    osVersion: str
    commands: Commands

@dataclass
class LogData:
    logMonoTime: int
    initData: InitData
    valid: bool
    # these just include a subset of the data

In [15]:
qlog_list: List[LogData] = list(qlog_data)

In [16]:
print([attr for attr in dir(qlog_list[0]) if not attr.startswith("__")])

['_get', '_get_by_field', '_has', '_has_by_field', '_parent', '_which', '_which_str', 'accelerometer', 'accelerometer2', 'androidGnssDEPRECATED', 'androidLog', 'applanixLocationDEPRECATED', 'applanixRawDEPRECATED', 'as_builder', 'boot', 'cameraOdometry', 'can', 'carControl', 'carParams', 'carState', 'cellInfoDEPRECATED', 'clocks', 'controlsState', 'customReserved0', 'customReserved1', 'customReserved2', 'customReserved3', 'customReserved4', 'customReserved5', 'customReserved6', 'customReserved7', 'customReserved8', 'customReserved9', 'customReservedRawData0', 'customReservedRawData1', 'customReservedRawData2', 'deviceState', 'driverCameraState', 'driverEncodeData', 'driverEncodeIdx', 'driverMonitoringState', 'driverStateDEPRECATED', 'driverStateV2', 'errorLogMessage', 'ethernetDataDEPRECATED', 'featuresDEPRECATED', 'gnssMeasurements', 'gpsLocation', 'gpsLocationExternal', 'gpsNMEA', 'gpsPlannerPlanDEPRECATED', 'gpsPlannerPointsDEPRECATED', 'gyroscope', 'gyroscope2', 'initData', 'is_roo

In [17]:
print(str(qlog_list[0].initData.params.entries[0].value))

b'{"alias": null, "athena_host": "prod-comma-public-athena-2.prod-comma-public-athena.production.svc.cluster.local", "device_type": "threex", "dongle_id": "12fdf6595e41b23a", "ignore_uploads": null, "last_athena_ping": 1710075958, "last_gps_accuracy": null, "last_gps_bearing": null, "last_gps_lat": null, "last_gps_lng": null, "last_gps_speed": null, "last_gps_time": null, "public_key": "-----BEGIN PUBLIC KEY-----\\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAzrLFx5osQs3GHw4xVPI1\\nx2UDMD7qQOvX+7WWH3wrA5tklpYCkL68fANGU5uK8LrRYRMbIVRRC+jheVDuqASq\\n7JOGGN+41r2nSVLfyd5dssj5ichyO/K6rziDPzryACMpToFpursx8qLELUTkwlqR\\nWmi1oYR9Z+Gu0FqMX/uTxa37UEIDzbqQPJM2t78+6FXxsu2NHobe8fAb/K3b1Kj6\\npWw+ven240VKSCb57CeX+qoqDNNwtLxDbJh1FqAMf6yhOrNksvVFPvdTrx+NKqlM\\ndpXnSs7LZjzAmyL85KHM+btpkO0wgJBzu10jwGdSSB7vtP4dYarjmEOOw2/3kLEg\\nqwIDAQAB\\n-----END PUBLIC KEY-----\\n", "serial": "8ff32d45", "sim_id": "", "trial_claimed": false, "is_paired": true, "eligible_features": {"prime": true, "prime_data": true, "n

In [18]:
qlog_list[0].to_dict()

{'initData': {'kernelArgs': ['rcupdate.rcu_expedited=1',
   'console=ttyMSM0,115200n8',
   'quiet',
   'loglevel=3',
   'earlycon=msm_geni_serial,0xA84000',
   'androidboot.hardware=qcom',
   'androidboot.console=ttyMSM0',
   'video=DSI-1:1080x2160@60e',
   'ehci-hcd.park=3',
   'lpm_levels.sleep_disabled=1',
   'service_locator.enable=1',
   'androidboot.selinux=permissive',
   'firmware_class.path=/lib/firmware/updates',
   'net.ifnames=0',
   'dyndbg=""',
   'root=/dev/sda7',
   'androidboot.bootdevice=1d84000.ufshc',
   'androidboot.serialno=8ff32d45',
   'androidboot.baseband=sda',
   'msm_drm.dsi_display0=dsi_fhd_ea8074_1080_cmd_display:',
   'androidboot.slot_suffix=_b',
   'skip_initramfs',
   'rootwait',
   'ro',
   'init=/sbin/init'],
  'dongleId': '12fdf6595e41b23a',
  'deviceType': 'tizi',
  'version': '0.9.6-release',
  'dirty': False,
  'gitCommit': 'e5d2ce51983f2b1b33fca929b7d4ec6c01e394da',
  'gitBranch': 'release3',
  'passive': False,
  'gitRemote': 'https://github.co

In [19]:
def isiterable(item):
    try:
        iter(item)
        return True
    except TypeError:
        return False
def get_fields(o, print_progress=True, depth=0):
    if isinstance(o, list) or isinstance(o, tuple) or isinstance(o, set) or isiterable(o):
        fields = []
        for item in o:
            field = get_fields(item, print_progress, depth=depth+1)
            fields.append(field)
        return fields
    elif isinstance(o, dict):
        total_dict = dict()
        for key, value in o.items():
            total_dict[key] = get_fields(value, print_progress, depth=depth+1)
        return total_dict
    elif isinstance(o, str) or isinstance(o, int) or isinstance(o, float) or isinstance(o, bool) or isinstance(o, bytes) or o is None:
        return type(o).__name__
    else:
        item_attrs = [attr for attr in dir(o) if not attr.startswith("_")]
        # print("item_attrs", item_attrs)
        total_dict = dict()
        progress = 0
        for attr in item_attrs:
            if print_progress:
                progress += 1
                print(f"Progress {depth}: {progress}/{len(item_attrs)} attr={attr}", end="\r")
            try:
                total_dict[attr] = get_fields(getattr(o, attr), print_progress, depth=depth+1)
            except Exception:
                continue
        return total_dict



In [20]:
import re

def parse_capnp_string(s: str):
    # Wrap words after = that are not numbers or true of false in quotes
    s = re.sub(r'(?<=\s=\s)(?!true|false|\d+)(\w+)', r'"\1"', s)
    # Wrap in quotes and replace = with :
    s = re.sub(r"(\w+) = ", r'"\1": ', s)

    # Replace ( with [ for lists
    pattern = r'\((?:"[^"]*"|\d+)(?:\s*,\s*(?:"[^"]*"|\d+))*\)'

    def convert(match):
        inner_text = match.group(0)[1:-1]  # Remove the outer parentheses
        return "[" + inner_text + "]"  # Surround with square brackets

    s = re.sub(pattern, convert, s)

    # Replace the rest of the parentheses with curly braces and remove newlines
    s = s.replace("(", "{").replace(")", "}").replace("\n", "")
    return s

In [21]:
# import json
# field_json = capnp_to_json(str(qlog_list[0]))
# field_json
# parse_custom_structure(str(qlog_list[0]))

In [22]:
qlog_list[0].to_dict()

{'initData': {'kernelArgs': ['rcupdate.rcu_expedited=1',
   'console=ttyMSM0,115200n8',
   'quiet',
   'loglevel=3',
   'earlycon=msm_geni_serial,0xA84000',
   'androidboot.hardware=qcom',
   'androidboot.console=ttyMSM0',
   'video=DSI-1:1080x2160@60e',
   'ehci-hcd.park=3',
   'lpm_levels.sleep_disabled=1',
   'service_locator.enable=1',
   'androidboot.selinux=permissive',
   'firmware_class.path=/lib/firmware/updates',
   'net.ifnames=0',
   'dyndbg=""',
   'root=/dev/sda7',
   'androidboot.bootdevice=1d84000.ufshc',
   'androidboot.serialno=8ff32d45',
   'androidboot.baseband=sda',
   'msm_drm.dsi_display0=dsi_fhd_ea8074_1080_cmd_display:',
   'androidboot.slot_suffix=_b',
   'skip_initramfs',
   'rootwait',
   'ro',
   'init=/sbin/init'],
  'dongleId': '12fdf6595e41b23a',
  'deviceType': 'tizi',
  'version': '0.9.6-release',
  'dirty': False,
  'gitCommit': 'e5d2ce51983f2b1b33fca929b7d4ec6c01e394da',
  'gitBranch': 'release3',
  'passive': False,
  'gitRemote': 'https://github.co

In [23]:
import base64
class BytesEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            # Encode bytes to a Base64 string
            return base64.b64encode(obj).decode('ascii')
        # Let the base class default method raise the TypeError
        return json.JSONEncoder.default(self, obj)

def capnp_to_json_valid_dict(obj):
    json_string = json.dumps(obj.to_dict(), cls=BytesEncoder)
    return json.loads(json_string)


In [24]:
qlog_dicts = [capnp_to_json_valid_dict(o) for o in qlog_list]

In [25]:
# jsons = [json.dumps(qlog_dict, indent=4) for qlog_dict in qlog_dicts]
qlog_name = "qlog" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".json"
with open(qlog_name, "w") as f:
    json.dump(qlog_dicts, f, indent=4)

In [26]:
for field in list(qlog_list[0].schema.node.struct.fields):
    print(field.name)

logMonoTime
initData
roadCameraState
gpsNMEA
sensorEventDEPRECATED
can
deviceState
controlsState
liveEventDEPRECATED
model
featuresDEPRECATED
sensorEventsDEPRECATED
pandaStateDEPRECATED
radarState
liveUIDEPRECATED
roadEncodeIdx
liveTracks
sendcan
logMessage
liveCalibration
androidLog
gpsLocation
carState
carControl
longitudinalPlan
liveLocationDEPRECATED
ethernetDataDEPRECATED
navUpdateDEPRECATED
cellInfoDEPRECATED
wifiScanDEPRECATED
androidGnssDEPRECATED
qcomGnss
lidarPtsDEPRECATED
procLog
ubloxGnss
clocks
liveMpcDEPRECATED
liveLongitudinalMpcDEPRECATED
navStatusDEPRECATED
ubloxRaw
gpsPlannerPointsDEPRECATED
gpsPlannerPlanDEPRECATED
applanixRawDEPRECATED
trafficEventsDEPRECATED
liveLocationTimingDEPRECATED
orbslamCorrectionDEPRECATED
liveLocationCorrectedDEPRECATED
orbObservationDEPRECATED
gpsLocationExternal
locationDEPRECATED
uiNavigationEventDEPRECATED
liveLocationKalmanDEPRECATED
testJoystick
orbOdometryDEPRECATED
orbFeaturesDEPRECATED
applanixLocationDEPRECATED
orbKeyFrameDEPRECA

In [27]:
result = parse_capnp_string(str(qlog_list[0]))
result

'{ "logMonoTime": 12308487556974,  "initData": {    "kernelArgs": [      "rcupdate.rcu_expedited=1",      "console=ttyMSM0,115200n8",      "quiet",      "loglevel=3",      "earlycon=msm_geni_serial,0xA84000",      "androidboot.hardware=qcom",      "androidboot.console=ttyMSM0",      "video=DSI-1:1080x2160@60e",      "ehci-hcd.park=3",      "lpm_levels.sleep_disabled=1",      "service_locator.enable=1",      "androidboot.selinux=permissive",      "firmware_class.path=/lib/firmware/updates",      "net.ifnames=0",      "dyndbg=\\"\\"",      "root=/dev/sda7",      "androidboot.bootdevice=1d84000.ufshc",      "androidboot.serialno=8ff32d45",      "androidboot.baseband=sda",      "msm_drm.dsi_display0=dsi_fhd_ea8074_1080_cmd_display:",      "androidboot.slot_suffix=_b",      "skip_initramfs",      "rootwait",      "ro",      "init=/sbin/init" ],    "dongleId": "12fdf6595e41b23a",    "deviceType": "tizi",    "version": "0.9.6-release",    "dirty": false,    "gitCommit": "e5d2ce51983f2b1b33fca

In [28]:
import json
#json.loads(result)

In [29]:
model_qlogs = [log for log in qlog_list if log.which() == "modelV2"]
len(model_qlogs)

30

In [30]:
print(model_qlogs[3])

( logMonoTime = 13517666270585,
  valid = true,
  modelV2 = (
    frameId = 24153,
    frameAge = 0,
    frameDropPerc = 0,
    timestampEof = 13517628680116,
    position = (
      x = [-0.93192095, -1.0267241, -0.10427952, 1.0354899, 2.6962252, 4.8163366, 7.510541, 10.582708, 14.27247, 18.283421, 22.91098, 28.020267, 33.559593, 39.631142, 46.168484, 53.051678, 60.84988, 68.615067, 76.987259, 86.055115, 95.518829, 105.41493, 115.83342, 126.65573, 137.79605, 150.04503, 162.54599, 174.85846, 188.54034, 203.03665, 216.81848, 231.74071, 247.8791],
      y = [0.23170662, 0.2243638, 0.2222946, 0.22634974, 0.23110768, 0.24416403, 0.27388722, 0.30917197, 0.3766886, 0.47657502, 0.62436467, 0.82731825, 1.0717885, 1.4327341, 1.8636618, 2.438884, 3.1281757, 3.9439929, 4.9468102, 6.1656981, 7.5350304, 9.1212187, 10.903283, 12.766989, 14.779788, 17.239473, 19.67494, 22.063877, 24.658119, 27.346087, 29.94976, 32.929691, 34.965855],
      z = [1.401997e-05, -5.2554264e-05, -0.00012424447, -0.00080631

# The different types of CAN messages

In [31]:
types = [item.which() for item in qlog_list]
len(types)

11367

In [32]:
qlog_types = set(types)
len(qlog_types)

44

In [33]:
qlog_types

{'accelerometer',
 'cameraOdometry',
 'can',
 'carControl',
 'carParams',
 'carState',
 'clocks',
 'controlsState',
 'deviceState',
 'driverCameraState',
 'driverEncodeIdx',
 'driverMonitoringState',
 'driverStateV2',
 'gpsLocation',
 'gyroscope',
 'initData',
 'liveCalibration',
 'liveLocationKalman',
 'liveParameters',
 'liveTorqueParameters',
 'longitudinalPlan',
 'magnetometer',
 'managerState',
 'mapRenderState',
 'microphone',
 'modelV2',
 'navInstruction',
 'navModel',
 'onroadEvents',
 'pandaStates',
 'peripheralState',
 'procLog',
 'qRoadEncodeIdx',
 'radarState',
 'roadCameraState',
 'roadEncodeIdx',
 'sendcan',
 'sentinel',
 'temperatureSensor',
 'thumbnail',
 'uiDebug',
 'uiPlan',
 'wideRoadCameraState',
 'wideRoadEncodeIdx'}

Which of these are the most interesting to look at? 


1. modelV2


# RLOG



In [34]:
rlog_data = LogReader(rlog_path.as_posix(), default_mode=ReadMode.RLOG)
rlog_list = list(rlog_data)
rlog_types = set([item.which() for item in rlog_list])

In [35]:
len(qlog_types), len(rlog_types)

(44, 49)

In [36]:
qlog_types == rlog_types

False

In [37]:
len(rlog_list)

69544

In [38]:
# Create rlog json file
from datetime import datetime
rlog_dicts = [capnp_to_json_valid_dict(o) for o in rlog_list]
name = "rlog" + datetime.now().strftime("%Y%m%d-%H%M%S") + ".json"
with open(name, "w") as f:
    json.dump(rlog_dicts, f, indent=4)

In [39]:
len(rlog_types)

49

In [40]:
rlog_types - qlog_types

{'androidLog', 'liveTracks', 'logMessage', 'navThumbnail', 'qcomGnss'}

In [41]:
rlog_types

{'accelerometer',
 'androidLog',
 'cameraOdometry',
 'can',
 'carControl',
 'carParams',
 'carState',
 'clocks',
 'controlsState',
 'deviceState',
 'driverCameraState',
 'driverEncodeIdx',
 'driverMonitoringState',
 'driverStateV2',
 'gpsLocation',
 'gyroscope',
 'initData',
 'liveCalibration',
 'liveLocationKalman',
 'liveParameters',
 'liveTorqueParameters',
 'liveTracks',
 'logMessage',
 'longitudinalPlan',
 'magnetometer',
 'managerState',
 'mapRenderState',
 'microphone',
 'modelV2',
 'navInstruction',
 'navModel',
 'navThumbnail',
 'onroadEvents',
 'pandaStates',
 'peripheralState',
 'procLog',
 'qRoadEncodeIdx',
 'qcomGnss',
 'radarState',
 'roadCameraState',
 'roadEncodeIdx',
 'sendcan',
 'sentinel',
 'temperatureSensor',
 'thumbnail',
 'uiDebug',
 'uiPlan',
 'wideRoadCameraState',
 'wideRoadEncodeIdx'}

In [42]:
# These types were recorded in the TrondheimShow dataset rlog
trondheim_show_rlog_types = set([
    "accelerometer",
    "androidLog",
    "cameraOdometry",
    "can",
    "carControl",
    "carParams",
    "carState",
    "controlsState",
    "deviceState",
    "driverCameraState",
    "driverEncodeIdx",
    "driverMonitoringState",
    "driverStateV2",
    "gpsLocation",
    "gyroscope",
    "initData",
    "lateralPlanDEPRECATED",
    "liveCalibration",
    "liveLocationKalman",
    "liveParameters",
    "liveTorqueParameters",
    "liveTracks",
    "logMessage",
    "longitudinalPlan",
    "magnetometer",
    "managerState",
    "mapRenderState",
    "microphone",
    "modelV2",
    "navInstruction",
    "navModel",
    "navThumbnail",
    "onroadEvents",
    "pandaStates",
    "peripheralState",
    "procLog",
    "qRoadEncodeIdx",
    "qcomGnss",
    "radarState",
    "roadCameraState",
    "roadEncodeIdx",
    "sentinel",
    "temperatureSensor",
    "thumbnail",
    "uiDebug",
    "uiPlan",
    "wideRoadCameraState",
    "wideRoadEncodeIdx"
])
rlog_types - trondheim_show_rlog_types

{'clocks', 'sendcan'}

In [43]:
trondheim_show_rlog_types - rlog_types

{'lateralPlanDEPRECATED'}

## Exploring the rlog data fields

In [44]:
# accelerometer
accelerometer = [log for log in rlog_list if log.which() == "accelerometer"]
accelerometer[0].accelerometer.to_dict()

{'acceleration': {'v': [10.13033390045166,
   0.1095721498131752,
   1.1214679479599],
  'status': 1},
 'version': 1,
 'sensor': 1,
 'type': 1,
 'timestamp': 13510124928140,
 'source': 'lsm6ds3trc',
 'uncalibratedDEPRECATED': False}

In [45]:
# androidLog
androidLog = [log for log in rlog_list if log.which() == "androidLog"]
androidLog[0].androidLog.to_dict()

{'id': 0,
 'ts': 1711641399293885,
 'priority': 7,
 'pid': 3643,
 'tid': 0,
 'tag': 'ModemManager',
 'message': '{"MESSAGE": "<dbg> [1711641399.293610] [modem0] running registration checks (CS: \'yes\', PS: \'yes\', EPS: \'yes\', 5GS: \'no\')", "PRIORITY": "7", "SYSLOG_FACILITY": "3", "SYSLOG_IDENTIFIER": "ModemManager", "SYSLOG_PID": "3643", "SYSLOG_RAW": "<31>Mar 28 16:56:39 ModemManager[3643]: <dbg> [1711641399.293610] [modem0] running registration checks (CS: \'yes\', PS: \'yes\', EPS: \'yes\', 5GS: \'no\')\\n", "SYSLOG_TIMESTAMP": "Mar 28 16:56:39 ", "_BOOT_ID": "c88aebd480fd4d5ca6351ac551eba55d", "_CAP_EFFECTIVE": "200000", "_CMDLINE": "/usr/sbin/ModemManager --filter-policy=strict --debug", "_COMM": "ModemManager", "_EXE": "/usr/sbin/ModemManager", "_GID": "0", "_HOSTNAME": "comma-8ff32d45", "_MACHINE_ID": "ee405918b0d4d243a3e0a97365c57120", "_PID": "3643", "_SELINUX_CONTEXT": "kernel", "_SOURCE_REALTIME_TIMESTAMP": "1711641399293732", "_SYSTEMD_CGROUP": "/system.slice/ModemMana

In [46]:
# cameraOdometry
cameraOdometry = [log for log in rlog_list if log.which() == "cameraOdometry"]
cameraOdometry[0].cameraOdometry.to_dict()

{'trans': [23.506019592285156, 0.0426732674241066, -0.018261700868606567],
 'rot': [-0.0023428304120898247, 0.008403465151786804, 0.036612577736377716],
 'transStd': [0.3209471106529236, 0.04810798913240433, 0.025323841720819473],
 'rotStd': [0.0026653213426470757,
  0.0023391209542751312,
  0.0011126782046630979],
 'frameId': 24003,
 'timestampEof': 13510128653401,
 'wideFromDeviceEuler': [4.62883326690644e-06,
  -0.006539277732372284,
  -0.02949448674917221],
 'wideFromDeviceEulerStd': [0.0047217016108334064,
  0.0030697225593030453,
  0.004497959744185209],
 'roadTransformTrans': [-3.6452547647058964e-08,
  7.799462764523923e-08,
  1.2459235191345215],
 'roadTransformTransStd': [0.0009840471902862191,
  0.000983926118351519,
  0.014705419540405273]}

In [47]:
# can
can = [log for log in rlog_list if log.which() == "can"]
can[0].to_dict()

{'can': [{'address': 688,
   'busTime': 0,
   'dat': b'\xf4\xff\x00\x07H',
   'src': 128},
  {'address': 897,
   'busTime': 0,
   'dat': b'\x80\x98@\x00\x00\xb4\x11\x05',
   'src': 0},
  {'address': 1056, 'busTime': 0, 'dat': b'\x10\x00I1e\x08i\x00', 'src': 0},
  {'address': 1057,
   'busTime': 0,
   'dat': b'\x00\x00\x00\xff\x03\x00\x00x',
   'src': 0},
  {'address': 897,
   'busTime': 0,
   'dat': b'\x80\x98@\x00\x00\xb4\x11\x05',
   'src': 128},
  {'address': 1056, 'busTime': 0, 'dat': b'\x10\x00I1e\x08i\x00', 'src': 128},
  {'address': 304,
   'busTime': 0,
   'dat': b'\xaf~\xde\x7f\x00\x00\x06\xee',
   'src': 0},
  {'address': 1057,
   'busTime': 0,
   'dat': b'\x00\x00\x00\xff\x03\x00\x00x',
   'src': 128},
  {'address': 320, 'busTime': 0, 'dat': b'\x9a\x80\x00` \x00\x06z', 'src': 0},
  {'address': 304,
   'busTime': 0,
   'dat': b'\xaf~\xde\x7f\x00\x00\x06\xee',
   'src': 128},
  {'address': 320,
   'busTime': 0,
   'dat': b'\x9a\x80\x00` \x00\x06z',
   'src': 128},
  {'address'

In [48]:
# carControl
carControl = [log for log in rlog_list if log.which() == "carControl"]
carControl[0].carControl.to_dict()

{'enabled': False,
 'gasDEPRECATED': 0.0,
 'brakeDEPRECATED': 0.0,
 'steeringTorqueDEPRECATED': 0.0,
 'cruiseControl': {'cancel': False,
  'resume': False,
  'speedOverrideDEPRECATED': 0.0,
  'accelOverrideDEPRECATED': 0.0,
  'override': False},
 'hudControl': {'speedVisible': False,
  'setSpeed': 70.83333587646484,
  'lanesVisible': False,
  'leadVisible': False,
  'visualAlert': 'none',
  'audibleAlert': 'none',
  'rightLaneVisible': True,
  'leftLaneVisible': True,
  'rightLaneDepart': False,
  'leftLaneDepart': False},
 'actuators': {'gas': 0.0,
  'brake': 0.0,
  'steer': -0.0,
  'steeringAngleDeg': 0.0,
  'accel': 0.0,
  'longControlState': 'off',
  'speed': 0.0,
  'curvature': 0.002037251368165016,
  'steerOutputCan': 0.0},
 'activeDEPRECATED': False,
 'rollDEPRECATED': 0.0,
 'pitchDEPRECATED': 0.0,
 'actuatorsOutput': {'gas': 0.0,
  'brake': 0.0,
  'steer': 0.0,
  'steeringAngleDeg': 0.0,
  'accel': 0.0,
  'longControlState': 'off',
  'speed': 0.0,
  'curvature': 0.0020372513681

In [49]:
# carParams
carParams = [log for log in rlog_list if log.which() == "carParams"]
carParams[0].carParams.to_dict()

{'carName': 'hyundai',
 'carFingerprint': 'HYUNDAI KONA ELECTRIC 2019',
 'enableGasInterceptor': False,
 'pcmCruise': True,
 'enableCameraDEPRECATED': False,
 'enableDsu': False,
 'enableApgsDEPRECATED': False,
 'minEnableSpeed': -1.0,
 'minSteerSpeed': 0.0,
 'safetyModelDEPRECATED': 'silent',
 'safetyParamDEPRECATED': 0,
 'mass': 1821.0,
 'wheelbase': 2.5999999046325684,
 'centerToFront': 1.0399999618530273,
 'steerRatio': 13.420000076293945,
 'steerRatioRear': 0.0,
 'rotationalInertia': 2887.498291015625,
 'tireStiffnessFront': 92143.28125,
 'tireStiffnessRear': 97106.5078125,
 'longitudinalTuning': {'kpBP': [0.0],
  'kpV': [0.5],
  'kiBP': [0.0],
  'kiV': [0.0],
  'deadzoneBP': [0.0],
  'deadzoneV': [0.0],
  'kf': 1.0},
 'lateralTuning': {'torque': {'useSteeringAngle': True,
   'kp': 1.0,
   'ki': 0.10000000149011612,
   'friction': 0.12359762191772461,
   'kf': 1.0,
   'steeringAngleDeadzoneDeg': 0.0,
   'latAccelFactor': 3.078814744949341,
   'latAccelOffset': 0.0}},
 'steerLimitA

In [50]:
# carState
carState = [log for log in rlog_list if log.which() == "carState"]
carState[0].carState.to_dict()

{'vEgo': 22.994779586791992,
 'wheelSpeeds': {'fl': 23.09895896911621,
  'fr': 23.02083396911621,
  'rl': 22.96875,
  'rr': 22.90798568725586},
 'gas': 0.17716535925865173,
 'gasPressed': True,
 'brake': 0.0,
 'brakePressed': False,
 'steeringAngleDeg': -1.2000000476837158,
 'steeringTorque': 24.0,
 'steeringPressed': False,
 'cruiseState': {'enabled': False,
  'speed': 0.0,
  'available': False,
  'speedOffset': 0.0,
  'standstill': False,
  'nonAdaptive': False,
  'speedCluster': 0.0},
 'events': [{'name': 'gasPressedOverride',
   'enable': False,
   'noEntry': False,
   'warning': False,
   'userDisable': False,
   'softDisable': False,
   'immediateDisable': False,
   'preEnable': False,
   'permanent': False,
   'overrideLongitudinal': True,
   'overrideLateral': False},
  {'name': 'wrongCarMode',
   'enable': False,
   'noEntry': True,
   'warning': False,
   'userDisable': True,
   'softDisable': False,
   'immediateDisable': False,
   'preEnable': False,
   'permanent': False,


In [51]:
# controlsState
controlsState = [log for log in rlog_list if log.which() == "controlsState"]
controlsState[0].controlsState.to_dict()

{'vEgoDEPRECATED': 0.0,
 'aEgoDEPRECATED': 0.0,
 'vPid': 22.994779586791992,
 'vTargetLead': 0.0,
 'upAccelCmd': 0.0,
 'uiAccelCmd': 0.0,
 'yActualDEPRECATED': 0.0,
 'yDesDEPRECATED': 0.0,
 'upSteerDEPRECATED': 0.0,
 'uiSteerDEPRECATED': 0.0,
 'aTargetMinDEPRECATED': 0.0,
 'aTargetMaxDEPRECATED': 0.0,
 'jerkFactorDEPRECATED': 0.0,
 'angleSteersDEPRECATED': 0.0,
 'hudLeadDEPRECATED': 0,
 'cumLagMs': 231.9042205810547,
 'canMonoTimeDEPRECATED': 0,
 'radarStateMonoTimeDEPRECATED': 0,
 'mdMonoTimeDEPRECATED': 0,
 'enabled': False,
 'steerOverrideDEPRECATED': False,
 'vCruise': 255.0,
 'rearViewCamDEPRECATED': False,
 'awarenessStatusDEPRECATED': 0.0,
 'angleModelBiasDEPRECATED': 0.0,
 'longitudinalPlanMonoTime': 13510070915537,
 'steeringAngleDesiredDegDEPRECATED': 0.0,
 'longControlState': 'off',
 'state': 'disabled',
 'vEgoRawDEPRECATED': 0.0,
 'ufAccelCmd': 0.0,
 'ufSteerDEPRECATED': 0.0,
 'aTarget': 0.0,
 'active': False,
 'curvature': 0.0016160709783434868,
 'alertStatus': 'normal',
 

In [52]:
# deviceState
deviceState = [log for log in rlog_list if log.which() == "deviceState"]
deviceState[0].deviceState.to_dict()

{'cpu0DEPRECATED': 0,
 'cpu1DEPRECATED': 0,
 'cpu2DEPRECATED': 0,
 'cpu3DEPRECATED': 0,
 'memDEPRECATED': 0,
 'gpuDEPRECATED': 0,
 'batDEPRECATED': 0,
 'freeSpacePercent': 10.000716209411621,
 'batteryPercentDEPRECATED': 0,
 'fanSpeedPercentDesired': 30,
 'started': True,
 'usbOnlineDEPRECATED': False,
 'startedMonoTime': 12308280824422,
 'thermalStatus': 'green',
 'batteryCurrentDEPRECATED': 0,
 'batteryVoltageDEPRECATED': 0,
 'chargingErrorDEPRECATED': False,
 'chargingDisabledDEPRECATED': False,
 'memoryUsagePercent': 58,
 'cpuUsagePercentDEPRECATED': 0,
 'pa0DEPRECATED': 0,
 'networkType': 'none',
 'offroadPowerUsageUwh': 0,
 'networkStrength': 'unknown',
 'carBatteryCapacityUwh': 30006240,
 'cpuTempC': [67.30000305175781,
  66.0,
  67.5999984741211,
  67.30000305175781,
  67.5999984741211,
  66.0,
  65.0,
  66.30000305175781],
 'gpuTempC': [68.9000015258789, 66.30000305175781],
 'memoryTempC': 60.79999923706055,
 'batteryTempCDEPRECATED': 0.0,
 'ambientTempCDEPRECATED': 0.0,
 'las

In [53]:
# driverCameraState
driverCameraState = [log for log in rlog_list if log.which() == "driverCameraState"]
driverCameraState[0].driverCameraState.to_dict()

{'frameId': 24002,
 'encodeId': 0,
 'timestampEof': 13510078878610,
 'frameLengthDEPRECATED': 0,
 'integLines': 175,
 'globalGainDEPRECATED': 0,
 'frameType': 'front',
 'timestampSof': 13510064096057,
 'lensPosDEPRECATED': 0,
 'lensSagDEPRECATED': 0.0,
 'lensErrDEPRECATED': 0.0,
 'lensTruePosDEPRECATED': 0.0,
 'gain': 7.320000171661377,
 'recoverStateDEPRECATED': 0,
 'highConversionGain': True,
 'measuredGreyFraction': 0.3125,
 'targetGreyFraction': 0.31340059638023376,
 'processingTime': 0.01497671939432621,
 'frameIdSensor': 0,
 'sensor': 'ox03c10',
 'exposureValPercent': 0.5619319081306458,
 'requestId': 24003}

In [54]:
# driverEncodeIdx
driverEncodeIdx = [log for log in rlog_list if log.which() == "driverEncodeIdx"]
driverEncodeIdx[0].driverEncodeIdx.to_dict()

{'frameId': 24002,
 'type': 'fullHEVC',
 'encodeId': 24000,
 'segmentNum': 20,
 'segmentId': 0,
 'segmentIdEncode': 0,
 'timestampSof': 13510064096057,
 'timestampEof': 13510078878610,
 'flags': 2147500040,
 'len': 153252}

In [55]:
# driverMonitoringState
driverMonitoringState = [log for log in rlog_list if log.which() == "driverMonitoringState"]
driverMonitoringState[6].driverMonitoringState.to_dict()

{'events': [],
 'faceDetected': True,
 'isDistracted': False,
 'awarenessStatus': 1.0,
 'isRHD': False,
 'rhdCheckedDEPRECATED': False,
 'posePitchOffset': -0.15810804069042206,
 'posePitchValidCount': 7200,
 'poseYawOffset': 0.12172555923461914,
 'poseYawValidCount': 7200,
 'stepChange': 0.004545454401522875,
 'awarenessActive': 1.0,
 'awarenessPassive': 1.0,
 'isLowStd': True,
 'hiStdCount': 0,
 'isPreviewDEPRECATED': False,
 'isActiveMode': True,
 'distractedType': 0}

In [56]:
# driverStateV2
driverMonitoringState = [log for log in rlog_list if log.which() == "driverStateV2"]
driverMonitoringState[0].driverStateV2.to_dict()


{'frameId': 24002,
 'modelExecutionTime': 0.020383542403578758,
 'dspExecutionTime': 0.019256822764873505,
 'rawPredictions': b'',
 'poorVisionProb': 0.0009158340399153531,
 'wheelOnRightProb': 0.0009158340399153531,
 'leftDriverData': {'faceOrientation': [0.1021132692694664,
   0.0,
   -0.1021132692694664],
  'faceOrientationStd': [0.04672863706946373,
   0.04672863706946373,
   0.029513493180274963],
  'facePosition': [0.14040574431419373, -0.0255283173173666],
  'facePositionStd': [0.002818605862557888, 0.0031216309871524572],
  'faceProb': 0.9633013010025024,
  'leftEyeProb': 0.8627166152000427,
  'rightEyeProb': 0.9206140041351318,
  'leftBlinkProb': 0.2748653292655945,
  'rightBlinkProb': 0.236078143119812,
  'sunglassesProb': 0.0009158340399153531,
  'occludedProb': 0.5,
  'readyProb': [0.5634764432907104,
   0.4872386157512665,
   0.44912010431289673,
   0.43652352690696716],
  'notReadyProb': [0.015741512179374695, 0.5127613544464111]},
 'rightDriverData': {'faceOrientation': 

In [57]:
# gpsLocation - removed in Orkdal set
# gpsLocation = [log for log in rlog_list if log.which() == "gpsLocation"]
# gpsLocation[0].gpsLocation.to_dict()

In [58]:
# gyroscope
gyroscope = [log for log in rlog_list if log.which() == "gyroscope"]
gyroscope[0].gyroscope.to_dict()

{'gyroUncalibrated': {'v': [-0.0507018156349659,
   -0.036651913076639175,
   0.029474249109625816],
  'status': 1},
 'version': 2,
 'sensor': 5,
 'type': 16,
 'timestamp': 13510124928140,
 'source': 'lsm6ds3trc',
 'uncalibratedDEPRECATED': False}

In [59]:
# initData
initData = [log for log in rlog_list if log.which() == "initData"]
initData[0].initData.to_dict()


{'kernelArgs': ['rcupdate.rcu_expedited=1',
  'console=ttyMSM0,115200n8',
  'quiet',
  'loglevel=3',
  'earlycon=msm_geni_serial,0xA84000',
  'androidboot.hardware=qcom',
  'androidboot.console=ttyMSM0',
  'video=DSI-1:1080x2160@60e',
  'ehci-hcd.park=3',
  'lpm_levels.sleep_disabled=1',
  'service_locator.enable=1',
  'androidboot.selinux=permissive',
  'firmware_class.path=/lib/firmware/updates',
  'net.ifnames=0',
  'dyndbg=""',
  'root=/dev/sda7',
  'androidboot.bootdevice=1d84000.ufshc',
  'androidboot.serialno=8ff32d45',
  'androidboot.baseband=sda',
  'msm_drm.dsi_display0=dsi_fhd_ea8074_1080_cmd_display:',
  'androidboot.slot_suffix=_b',
  'skip_initramfs',
  'rootwait',
  'ro',
  'init=/sbin/init'],
 'dongleId': '12fdf6595e41b23a',
 'deviceType': 'tizi',
 'version': '0.9.6-release',
 'dirty': False,
 'gitCommit': 'e5d2ce51983f2b1b33fca929b7d4ec6c01e394da',
 'gitBranch': 'release3',
 'passive': False,
 'gitRemote': 'https://github.com/commaai/openpilot.git',
 'kernelVersion': '

In [60]:
# lateralPlanDEPRECATED - removed in Orkdal set
# lateralPlanDEPRECATED = [log for log in rlog_list if log.which() == "lateralPlanDEPRECATED"]
# lateralPlanDEPRECATED[0].lateralPlanDEPRECATED.to_dict()

In [61]:
# liveCalibration
liveCalibration = [log for log in rlog_list if log.which() == "liveCalibration"]
liveCalibration[0].liveCalibration.to_dict()

{'calStatusDEPRECATED': 0,
 'calCycle': 0,
 'calPerc': 100,
 'rpyCalib': [-0.00010409663082100451,
  0.06890719383955002,
  -0.00685680890455842],
 'rpyCalibSpread': [0.0004718953568954021,
  0.005037431139498949,
  0.006849417928606272],
 'validBlocks': 50,
 'wideFromDeviceEuler': [1.4408644233299128e-07,
  -0.006299655418843031,
  -0.025989456102252007],
 'calStatus': 'calibrated',
 'height': [1.247731328010559]}

In [62]:
# liveLocationKalman
liveLocationKalman = [log for log in rlog_list if log.which() == "liveLocationKalman"]
liveLocationKalman[0].liveLocationKalman.to_dict()

{'positionECEF': {'value': [2812874.5194319515,
   524251.20124631654,
   5681315.557729948],
  'std': [0.8668089260268139, 0.9031301018394179, 0.8613821264691095],
  'valid': True},
 'positionGeodetic': {'value': [63.42285512159836,
   10.55740770346718,
   105.5716744649671],
  'std': [nan, nan, nan],
  'valid': True},
 'velocityECEF': {'value': [10.02746611663606,
   -20.617546163895618,
   -3.4983724522414597],
  'std': [0.13990115148187765, 0.1774783491909942, 0.13660024379365332],
  'valid': True},
 'velocityNED': {'value': [-7.002681389055708,
   -22.10582388324388,
   0.4085341594869645],
  'std': [nan, nan, nan],
  'valid': True},
 'velocityDevice': {'value': [23.132159036888236,
   -0.13638740213450462,
   -1.6602750145388825],
  'std': [0.1820581398962984, 0.10700262760046769, 0.1394022842980419],
  'valid': True},
 'accelerationDevice': {'value': [0.18843549497701664,
   0.6125345382837534,
   -0.14489076208626772],
  'std': [0.3357542022096994, 0.33617890078895585, 0.33266

In [63]:
# liveParameters
liveParameters = [log for log in rlog_list if log.which() == "liveParameters"]
liveParameters[0].liveParameters.to_dict()

{'valid': True,
 'gyroBias': 0.0,
 'angleOffsetDeg': 1.5494959354400635,
 'angleOffsetAverageDeg': 1.8157869577407837,
 'stiffnessFactor': 1.0164830684661865,
 'steerRatio': 12.677591323852539,
 'sensorValid': True,
 'yawRateDEPRECATED': 0.0,
 'posenetSpeed': 0.0,
 'posenetValid': True,
 'angleOffsetFastStd': 0.006249196827411652,
 'angleOffsetAverageStd': 0.003710170043632388,
 'stiffnessFactorStd': 0.0074362135492265224,
 'steerRatioStd': 0.09458543360233307,
 'roll': 0.09726572781801224}

In [64]:
# liveTorqueParameters
liveTorqueParameters = [log for log in rlog_list if log.which() == "liveTorqueParameters"]
liveTorqueParameters[0].liveTorqueParameters.to_dict()

{'liveValid': False,
 'latAccelFactorRaw': 3.1898961067199707,
 'latAccelOffsetRaw': -0.10439243912696838,
 'frictionCoefficientRaw': 0.08201266825199127,
 'latAccelFactorFiltered': 3.078814744949341,
 'latAccelOffsetFiltered': 0.0,
 'frictionCoefficientFiltered': 0.12359762191772461,
 'totalBucketPoints': 4830.0,
 'decay': 50.0,
 'maxResets': 1.0,
 'version': 1,
 'useParams': True}

In [65]:
# liveTracks
liveTracks = [log for log in rlog_list if log.which() == "liveTracks"]
liveTracks[5].liveTracks

<capnp list reader []>

In [66]:
# logMessage
logMessage = [log for log in rlog_list if log.which() == "logMessage"]
logMessage[0].logMessage

'{"created": 1711641399.4105115, "ctx": {"branch": "release3", "commit": "e5d2ce51983f2b1b33fca929b7d4ec6c01e394da", "daemon": "loggerd", "device": "tizi", "dirty": false, "dongle_id": "12fdf6595e41b23a", "origin": "github.com/commaai/openpilot", "version": "0.9.6-release"}, "filename": "system/loggerd/loggerd.cc", "funcname": "handle_encoder_msg", "levelnum": 10, "lineno": 161, "msg": "rotate 19 -> 20 ready 4/4 for driverEncodeData"}'

In [67]:
# longitudinalPlan
longitudinalPlan = [log for log in rlog_list if log.which() == "longitudinalPlan"]
longitudinalPlan[0].longitudinalPlan.to_dict()


{'lateralValidDEPRECATED': False,
 'longitudinalValidDEPRECATED': False,
 'vTargetDEPRECATED': 0.0,
 'aTargetMinDEPRECATED': 0.0,
 'aTargetMaxDEPRECATED': 0.0,
 'jerkFactorDEPRECATED': 0.0,
 'hasLead': False,
 'fcw': False,
 'modelMonoTime': 13510115180172,
 'radarStateMonoTimeDEPRECATED': 0,
 'laneWidthDEPRECATED': 0.0,
 'vTargetFutureDEPRECATED': 0.0,
 'longitudinalPlanSource': 'cruise',
 'vCruiseDEPRECATED': 0.0,
 'aCruiseDEPRECATED': 0.0,
 'aTargetDEPRECATED': 0.0,
 'gpsPlannerActiveDEPRECATED': False,
 'vMaxDEPRECATED': 0.0,
 'vCurvatureDEPRECATED': 0.0,
 'decelForTurnDEPRECATED': False,
 'hasLeftLaneDEPRECATED': False,
 'hasRightLaneDEPRECATED': False,
 'mapValidDEPRECATED': False,
 'vStartDEPRECATED': 0.0,
 'aStartDEPRECATED': 0.0,
 'radarValidDEPRECATED': False,
 'processingDelay': -13510115328000.0,
 'radarCanErrorDEPRECATED': False,
 'commIssueDEPRECATED': False,
 'accels': [0.008285567164421082,
  0.020646624267101288,
  0.05772979557514191,
  0.11699613928794861,
  0.194114

In [68]:
# magnetometer
magnetometer = [log for log in rlog_list if log.which() == "magnetometer"]
magnetometer[0].magnetometer.to_dict()

{'magneticUncalibrated': {'v': [-0.25494384765625,
   0.37286376953125,
   0.72076416015625,
   0.48095703125,
   -1.11492919921875,
   0.782958984375],
  'status': 1},
 'version': 1,
 'sensor': 3,
 'type': 14,
 'timestamp': 13510102412620,
 'source': 'mmc5603nj',
 'uncalibratedDEPRECATED': False}

In [69]:
# managerState
managerState = [log for log in rlog_list if log.which() == "managerState"]
managerState[0].managerState.to_dict()

{'processes': [{'name': 'manage_athenad',
   'pid': 0,
   'running': False,
   'exitCode': 0,
   'shouldBeRunning': False},
  {'name': 'camerad',
   'pid': 96286,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'logcatd',
   'pid': 96287,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'proclogd',
   'pid': 96288,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'logmessaged',
   'pid': 17559,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'micd',
   'pid': 96290,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'timed',
   'pid': 17592,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'dmonitoringmodeld',
   'pid': 96291,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'encoderd',
   'pid': 96292,
   'running': True,
   'exitCode': 0,
   'shouldBeRunning': True},
  {'name': 'strea

In [70]:
# mapRenderState
mapRenderState = [log for log in rlog_list if log.which() == "mapRenderState"]
mapRenderState[0].mapRenderState.to_dict()

{'locationMonoTime': 13510518297620, 'renderTime': 0.0, 'frameId': 2397}

In [71]:
# microphone
microphone = [log for log in rlog_list if log.which() == "microphone"]
microphone[0].microphone.to_dict()

{'soundPressure': 0.05908488854765892,
 'soundPressureWeightedDb': 47.229652404785156,
 'filteredSoundPressureWeightedDb': 0.0,
 'soundPressureWeighted': 0.004597403109073639}

In [72]:
# modelV2
modelV2 = [log for log in rlog_list if log.which() == "modelV2"]
modelV2[0].modelV2.to_dict()


{'frameId': 24003,
 'frameAge': 0,
 'frameDropPerc': 0.0,
 'timestampEof': 13510128653401,
 'position': {'x': [-0.5142419934272766,
   -0.4453819990158081,
   0.3868841528892517,
   1.5110664367675781,
   3.109633207321167,
   5.166661739349365,
   7.6662445068359375,
   10.660078048706055,
   14.006183624267578,
   18.034584045410156,
   22.298484802246094,
   27.187538146972656,
   32.4564323425293,
   38.10005187988281,
   44.25063705444336,
   51.015899658203125,
   58.108802795410156,
   65.32728576660156,
   73.21658325195312,
   81.94824981689453,
   90.62806701660156,
   100.14146423339844,
   109.74310302734375,
   119.75818634033203,
   130.4336700439453,
   141.06472778320312,
   152.74484252929688,
   164.14157104492188,
   176.36859130859375,
   188.56076049804688,
   201.8001708984375,
   214.3126983642578,
   228.27464294433594],
  'y': [0.08734115958213806,
   0.08900663256645203,
   0.09431838989257812,
   0.09959644079208374,
   0.1162981390953064,
   0.13519878685474

In [73]:
# navInstruction
navInstruction = [log for log in rlog_list if log.which() == "navInstruction"]
navInstruction[0].navInstruction.to_dict()

{'maneuverDistance': 0.0,
 'distanceRemaining': 0.0,
 'timeRemaining': 0.0,
 'timeRemainingTypical': 0.0,
 'showFull': False,
 'speedLimit': 0.0,
 'speedLimitSign': 'mutcd'}

In [74]:
# navModel
navModel = [log for log in rlog_list if log.which() == "navModel"]
navModel[0].navModel.to_dict()

{'frameId': 2397,
 'modelExecutionTime': 0.00791484396904707,
 'dspExecutionTime': 0.007808541879057884,
 'features': [0.4331595301628113,
  0.41855862736701965,
  0.438026487827301,
  0.3942238390445709,
  0.41855862736701965,
  0.4915630519390106,
  0.4672282636165619,
  0.38448992371559143,
  0.47209522128105164,
  0.4672282636165619,
  0.5158978700637817,
  0.438026487827301,
  0.2628158926963806,
  0.4331595301628113,
  0.3893568813800812,
  0.36502206325531006,
  0.44289344549179077,
  0.37475597858428955,
  0.4039577543735504,
  0.4234256148338318,
  0.39909079670906067,
  0.36502206325531006,
  0.45749431848526,
  0.3893568813800812,
  0.438026487827301,
  0.4331595301628113,
  0.4331595301628113,
  0.438026487827301,
  0.42829257249832153,
  0.4331595301628113,
  0.3942238390445709,
  0.3942238390445709,
  0.438026487827301,
  0.4477604031562805,
  0.3796229660511017,
  0.4769621789455414,
  0.46236127614974976,
  0.5158978700637817,
  0.36502206325531006,
  0.3552881479263305

In [75]:
# navThumbnail
navThumbnail = [log for log in rlog_list if log.which() == "navThumbnail"]
navThumbnail[0].navThumbnail.to_dict()

{'frameId': 2400,
 'timestampEof': 13512026948244,
 'thumbnail': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00d\x00d\x00\x00\xff\xdb\x00C\x00\x10\x0b\x0c\x0e\x0c\n\x10\x0e\r\x0e\x12\x11\x10\x13\x18(\x1a\x18\x16\x16\x181#%\x1d(:3=<9387@H\\N@DWE78PmQW_bghg>Mqypdx\\egc\xff\xdb\x00C\x01\x11\x12\x12\x18\x15\x18/\x1a\x1a/cB8Bcccccccccccccccccccccccccccccccccccccccccccccccccc\xff\xc0\x00\x11\x08\x01\x00\x01\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\

In [76]:
# onroadEvents
onroadEvents = [log for log in rlog_list if log.which() == "onroadEvents"]
onroadEvents[0].onroadEvents[0].to_dict()

{'name': 'gasPressedOverride',
 'enable': False,
 'noEntry': False,
 'warning': False,
 'userDisable': False,
 'softDisable': False,
 'immediateDisable': False,
 'preEnable': False,
 'permanent': False,
 'overrideLongitudinal': True,
 'overrideLateral': False}

In [77]:
# pandaStates
pandaStates = [log for log in rlog_list if log.which() == "pandaStates"]
pandaStates[0].pandaStates[0].to_dict()

{'voltage': 13596,
 'current': 0,
 'ignitionLine': True,
 'controlsAllowed': False,
 'gasInterceptorDetectedDEPRECATED': False,
 'startedSignalDetectedDEPRECATED': False,
 'hasGpsDEPRECATED': False,
 'rxBufferOverflow': 0,
 'txBufferOverflow': 0,
 'gmlanSendErrs': 0,
 'pandaType': 'tres',
 'fanSpeedRpmDEPRECATED': 0,
 'usbPowerModeDEPRECATED': 'none',
 'ignitionCan': False,
 'safetyModel': 'hyundai',
 'faultStatus': 'none',
 'powerSaveEnabled': False,
 'uptime': 13545,
 'faults': [],
 'safetyRxInvalid': 0,
 'safetyParamDEPRECATED': 0,
 'harnessStatus': 'normal',
 'heartbeatLost': False,
 'alternativeExperience': 1,
 'safetyTxBlocked': 0,
 'interruptLoad': 0.15795397758483887,
 'safetyParam2DEPRECATED': 0,
 'safetyParam': 65,
 'fanPower': 23,
 'canState0': {'busOff': False,
  'busOffCnt': 0,
  'errorWarning': False,
  'errorPassive': False,
  'lastError': 'noError',
  'lastStoredError': 'stuffError',
  'lastDataError': 'noChange',
  'lastDataStoredError': 'noError',
  'receiveErrorCnt':

In [78]:
# peripheralState
peripheralState = [log for log in rlog_list if log.which() == "peripheralState"]
peripheralState[0].peripheralState.to_dict()

{'pandaType': 'tres',
 'voltage': 13883,
 'current': 549,
 'fanSpeedRpm': 1983,
 'usbPowerModeDEPRECATED': 'none'}

In [79]:
# procLog
procLog = [log for log in rlog_list if log.which() == "procLog"]
procLog[0].procLog.to_dict()

{'cpuTimes': [{'cpuNum': 0,
   'user': 2145.239990234375,
   'nice': 0.03999999910593033,
   'system': 1589.3800048828125,
   'idle': 8602.900390625,
   'iowait': 28.959999084472656,
   'irq': 156.10000610351562,
   'softirq': 81.76000213623047},
  {'cpuNum': 1,
   'user': 1722.9100341796875,
   'nice': 0.05999999865889549,
   'system': 1673.1099853515625,
   'idle': 8684.669921875,
   'iowait': 31.06999969482422,
   'irq': 480.20001220703125,
   'softirq': 43.02000045776367},
  {'cpuNum': 2,
   'user': 1752.3499755859375,
   'nice': 0.009999999776482582,
   'system': 937.72998046875,
   'idle': 10213.169921875,
   'iowait': 19.809999465942383,
   'irq': 52.66999816894531,
   'softirq': 27.989999771118164},
  {'cpuNum': 3,
   'user': 1320.56005859375,
   'nice': 0.009999999776482582,
   'system': 1838.239990234375,
   'idle': 9790.33984375,
   'iowait': 14.699999809265137,
   'irq': 42.689998626708984,
   'softirq': 26.510000228881836},
  {'cpuNum': 4,
   'user': 2149.139892578125,
   

In [80]:
# qRoadEncodeIdx
qRoadEncodeIdx = [log for log in rlog_list if log.which() == "qRoadEncodeIdx"]
qRoadEncodeIdx[0].qRoadEncodeIdx.to_dict()

{'frameId': 24002,
 'type': 'qcameraH264',
 'encodeId': 24000,
 'segmentNum': 20,
 'segmentId': 0,
 'segmentIdEncode': 0,
 'timestampSof': 13510063973662,
 'timestampEof': 13510078676162,
 'flags': 2147500040,
 'len': 2992}

In [81]:
# qcomGnss
qcomGnss = [log for log in rlog_list if log.which() == "qcomGnss"]
qcomGnss[0].qcomGnss.to_dict()

{'measurementReport': {'source': 'glonass',
  'fCount': 9632354,
  'gpsWeek': 0,
  'glonassCycleNumber': 8,
  'glonassNumberOfDays': 88,
  'milliseconds': 64600396,
  'timeBias': -0.12739069759845734,
  'clockTimeUncertainty': 13.643451690673828,
  'clockFrequencyBias': 24.445642471313477,
  'clockFrequencyUncertainty': 1.6059801578521729,
  'sv': [{'svId': 74,
    'glonassFrequencyIndex': -7,
    'observationState': 'track',
    'observations': 74,
    'goodObservations': 74,
    'gpsParityErrorCount': 0,
    'glonassHemmingErrorCount': 0,
    'filterStages': 1,
    'carrierNoise': 2790,
    'latency': 8,
    'predetectInterval': 20,
    'postdetections': 46,
    'unfilteredMeasurementIntegral': 64600319,
    'unfilteredMeasurementFraction': 0.7656766176223755,
    'unfilteredTimeUncertainty': 0.00020777707686647773,
    'unfilteredSpeed': 100.86177825927734,
    'unfilteredSpeedUncertainty': 0.5099999904632568,
    'measurementStatus': {'subMillisecondIsValid': True,
     'subBitTime

In [82]:
# radarState
radarState = [log for log in rlog_list if log.which() == "radarState"]
radarState[0].radarState.to_dict()


{'angleOffsetDEPRECATED': 0.0,
 'calStatusDEPRECATED': 0,
 'leadOne': {'dRel': 0.0,
  'yRel': 0.0,
  'vRel': 0.0,
  'aRel': 0.0,
  'vLead': 0.0,
  'aLeadDEPRECATED': 0.0,
  'dPath': 0.0,
  'vLat': 0.0,
  'vLeadK': 0.0,
  'aLeadK': 0.0,
  'fcw': False,
  'status': False,
  'aLeadTau': 0.0,
  'modelProb': 0.0,
  'radar': False,
  'radarTrackId': -1},
 'leadTwo': {'dRel': 0.0,
  'yRel': 0.0,
  'vRel': 0.0,
  'aRel': 0.0,
  'vLead': 0.0,
  'aLeadDEPRECATED': 0.0,
  'dPath': 0.0,
  'vLat': 0.0,
  'vLeadK': 0.0,
  'aLeadK': 0.0,
  'fcw': False,
  'status': False,
  'aLeadTau': 0.0,
  'modelProb': 0.0,
  'radar': False,
  'radarTrackId': -1},
 'cumLagMs': 178.8976593017578,
 'mdMonoTime': 13510115180172,
 'ftMonoTimeDEPRECATED': 0,
 'calCycleDEPRECATED': 0,
 'calPercDEPRECATED': 0,
 'carStateMonoTime': 13510143939078,
 'radarErrors': []}

In [83]:
# roadCameraState
roadCameraState = [log for log in rlog_list if log.which() == "roadCameraState"]
roadCameraState[0].roadCameraState.to_dict()

{'frameId': 24003,
 'encodeId': 0,
 'timestampEof': 13510128653401,
 'frameLengthDEPRECATED': 0,
 'integLines': 47,
 'globalGainDEPRECATED': 0,
 'frameType': 'unknown',
 'timestampSof': 13510113951578,
 'lensPosDEPRECATED': 0,
 'lensSagDEPRECATED': 0.0,
 'lensErrDEPRECATED': 0.0,
 'lensTruePosDEPRECATED': 0.0,
 'gain': 7.320000171661377,
 'recoverStateDEPRECATED': 0,
 'highConversionGain': True,
 'measuredGreyFraction': 0.35546875,
 'targetGreyFraction': 0.3522370159626007,
 'processingTime': 0.005936094094067812,
 'frameIdSensor': 0,
 'sensor': 'ox03c10',
 'exposureValPercent': 0.14910002052783966,
 'requestId': 24004}

In [84]:
# roadEncodeIdx
roadEncodeIdx = [log for log in rlog_list if log.which() == "roadEncodeIdx"]
roadEncodeIdx[0].roadEncodeIdx.to_dict()

{'frameId': 24002,
 'type': 'fullHEVC',
 'encodeId': 24000,
 'segmentNum': 20,
 'segmentId': 0,
 'segmentIdEncode': 0,
 'timestampSof': 13510063973662,
 'timestampEof': 13510078676162,
 'flags': 2147500040,
 'len': 149879}

In [85]:
# sentinel
sentinel = [log for log in rlog_list if log.which() == "sentinel"]
sentinel[0].sentinel.to_dict()

{'type': 'startOfSegment', 'signal': 0}

In [86]:
# temperatureSensor
temperatureSensor = [log for log in rlog_list if log.which() == "temperatureSensor"]
temperatureSensor[0].temperatureSensor.to_dict()

{'temperature': 54.79296875,
 'version': 1,
 'sensor': 0,
 'type': 13,
 'timestamp': 13510248347255,
 'source': 'lsm6ds3trc',
 'uncalibratedDEPRECATED': False}

In [87]:
# thumbnail
thumbnail = [log for log in rlog_list if log.which() == "thumbnail"]
thumbnail[0].thumbnail.to_dict()


{'frameId': 24003,
 'timestampEof': 13510128653401,
 'thumbnail': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x10\x0b\x0c\x0e\x0c\n\x10\x0e\r\x0e\x12\x11\x10\x13\x18(\x1a\x18\x16\x16\x181#%\x1d(:3=<9387@H\\N@DWE78PmQW_bghg>Mqypdx\\egc\xff\xdb\x00C\x01\x11\x12\x12\x18\x15\x18/\x1a\x1a/cB8Bcccccccccccccccccccccccccccccccccccccccccccccccccc\xff\xc0\x00\x11\x08\x01.\x01\xe2\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\

In [88]:
# uiDebug
uiDebug = [log for log in rlog_list if log.which() == "uiDebug"]
uiDebug[0].uiDebug.to_dict()

{'drawTimeMillis': 3.0235939025878906}

In [89]:
# uiPlan
uiPlan = [log for log in rlog_list if log.which() == "uiPlan"]
uiPlan[0].uiPlan.to_dict()

{'position': {'x': [-0.43510371446609497,
   -0.3489222228527069,
   0.46937867999076843,
   1.5982224941253662,
   3.2059555053710938,
   5.269461154937744,
   7.782255172729492,
   10.789691925048828,
   14.14765739440918,
   18.1716365814209,
   22.473358154296875,
   27.351764678955078,
   32.661224365234375,
   38.34092712402344,
   44.50590515136719,
   51.2894287109375,
   58.40083312988281,
   65.67428588867188,
   73.60384368896484,
   82.39338684082031,
   91.12753295898438,
   100.65158081054688,
   110.376220703125,
   120.36183166503906,
   131.14463806152344,
   141.90216064453125,
   153.64990234375,
   165.12796020507812,
   177.3775634765625,
   189.75814819335938,
   203.20101928710938,
   215.57627868652344,
   229.88087463378906],
  'y': [0.03493054583668709,
   0.036556705832481384,
   0.0418744832277298,
   0.04615592956542969,
   0.06301035732030869,
   0.08345852792263031,
   0.13707071542739868,
   0.18638694286346436,
   0.2826041877269745,
   0.40909779071807

In [90]:
# wideRoadCameraState
wideRoadCameraState = [log for log in rlog_list if log.which() == "wideRoadCameraState"]
wideRoadCameraState[0].wideRoadCameraState.to_dict()

{'frameId': 24003,
 'encodeId': 0,
 'timestampEof': 13510128787203,
 'frameLengthDEPRECATED': 0,
 'integLines': 81,
 'globalGainDEPRECATED': 0,
 'frameType': 'unknown',
 'timestampSof': 13510114031526,
 'lensPosDEPRECATED': 0,
 'lensSagDEPRECATED': 0.0,
 'lensErrDEPRECATED': 0.0,
 'lensTruePosDEPRECATED': 0.0,
 'gain': 8.235000610351562,
 'recoverStateDEPRECATED': 0,
 'highConversionGain': True,
 'measuredGreyFraction': 0.33984375,
 'targetGreyFraction': 0.33868706226348877,
 'processingTime': 0.010285415686666965,
 'frameIdSensor': 0,
 'sensor': 'ox03c10',
 'exposureValPercent': 0.2867106795310974,
 'requestId': 24004}

In [91]:
# wideRoadEncodeIdx
wideRoadEncodeIdx = [log for log in rlog_list if log.which() == "wideRoadEncodeIdx"]
wideRoadEncodeIdx[0].wideRoadEncodeIdx.to_dict()

{'frameId': 24002,
 'type': 'fullHEVC',
 'encodeId': 24000,
 'segmentNum': 20,
 'segmentId': 0,
 'segmentIdEncode': 0,
 'timestampSof': 13510064048610,
 'timestampEof': 13510078802620,
 'flags': 2147500040,
 'len': 108766}

In [92]:
# clocks
clocks = [log for log in rlog_list if log.which() == "clocks"]
clocks[0].clocks.to_dict()

{'bootTimeNanosDEPRECATED': 0,
 'monotonicNanosDEPRECATED': 0,
 'monotonicRawNanosDEPRECATD': 0,
 'wallTimeNanos': 1711641406593771141,
 'modemUptimeMillisDEPRECATED': 0}

In [93]:
# sendcan
sendcan = [log for log in rlog_list if log.which() == "sendcan"]
sendcan[0].to_dict()

{'sendcan': [{'address': 832,
   'busTime': 0,
   'dat': b'\x13\x00\x00$\xe4\x00,\x11',
   'src': 0}],
 'logMonoTime': 13510121503557,
 'valid': True}

In [94]:
# errorLogMessage
errorLogMessage = [log for log in rlog_list if log.which() == "errorLogMessage"]
errorLogMessage[0].errorLogMessage

IndexError: list index out of range

# NEW

In [ ]:
# Huh, the rlog.json and qlog.json contains NaN values, but the loading doesn't crash because of it...
with open("rlog.json", "r") as f:
    rlog_json = json.load(f)
rlog_json

FileNotFoundError: [Errno 2] No such file or directory: 'rlog.json'

# The difference between rlog and qlog

I am not entirely sure, but ChatGPT claims rlog is the Raw Logs, while the qlog is the quantized logs.
This means that qlog is a simplified version of the raw data. 

This is likely true as the original qlog file is much smaller than the rlog file. The json file for qlog is 20.8 MB while rlog.json is over 200 MB.